In [1]:
import pickle
import pandas as pd
import numpy as np
import joblib

test = pd.read_excel ('data/test1.xlsx')

#autoMachineLearning
import h2o
from h2o.automl import H2OAutoML

In [2]:
test

,average_temp,total_precip_mm,did_rain,is_closed,is_lockdown,is_curfew,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,...,year_2019.0,year_2020.0,year_2021.0,day_type_domingo,day_type_festivo,day_type_laborable,day_type_sábado,prev_sales,is_post_holiday,is_pre_holiday
0,23,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,723,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,1008,0,0
2,25,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,1012,0,1
3,23,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,2020,0,0
4,24,0,0,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,6700,1,0


In [3]:
# load the model from disk
loaded_model = pickle.load(open("models/best_rf.pkl", 'rb'))


In [4]:
loaded_model.predict(test)

array([1022.32022524, 1175.92904385, 1863.35556586, 5288.63866618,
       3759.58067986])

In [5]:
h2o.init() #To start h2o


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /home/fran/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpf_yuiosp
  JVM stdout: /tmp/tmpf_yuiosp/h2o_fran_started_from_python.out
  JVM stderr: /tmp/tmpf_yuiosp/h2o_fran_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_fran_onigox
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.109 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [6]:
saved_model = h2o.load_model('/mnt/c/Users/lesto/Desktop/Ironhack/CityPlayForecast/models/deeplearning/DeepLearning_grid__2_AutoML_20210515_173143_model_1')

In [7]:
stacked_test = test
h2test_stacked = h2o.H2OFrame(stacked_test) #Conversion into a H20 frame to train
h2test_stacked.head() #preview

Parse progress: |█████████████████████████████████████████████████████████| 100%


average_temp,total_precip_mm,did_rain,is_closed,is_lockdown,is_curfew,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,month_name_April,month_name_August,month_name_December,month_name_February,month_name_January,month_name_July,month_name_June,month_name_March,month_name_May,month_name_November,month_name_October,month_name_September,year_2015.0,year_2016.0,year_2017.0,year_2018.0,year_2019.0,year_2020.0,year_2021.0,day_type_domingo,day_type_festivo,day_type_laborable,day_type_sábado,prev_sales,is_post_holiday,is_pre_holiday
23,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,723,0,0
25,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1008,0,0
25,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1012,0,1
23,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,2020,0,0
24,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,6700,1,0


In [8]:
predicted_price_h2_stacked = saved_model.predict(h2test_stacked).as_data_frame() #PREDICTING the Sales on the TEST dataset
predicted_price_h2_stacked #Result

deeplearning prediction progress: |███████████████████████████████████████| 100%


,predict
0,1091.697634
1,1380.062215
2,2748.804550
3,7574.141831
4,3116.489529
